<a href="https://colab.research.google.com/github/TingfangHuang/GenAIHumanities/blob/main/%E9%BB%83%E8%8E%9B%E9%88%81%EF%BC%BF61221055L_HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW5 RAG

## 1. 請完成以下程式 (50%)

以下程式碼有許多不完整的地方，請依照原本的程式框架，補齊所有未完成的部分，請確保所有程式碼能完整執行並得到合理的回答結果。

In [ ]:
# ✅ STEP 1: 安裝需要的套件

!pip install -q sentence-transformers faiss-cpu requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00


In [ ]:
# ✅ STEP 2: 載入必要模組

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import pandas as pd

In [ ]:
### 非台大學生請執行以下程式碼獲取語料 ###

!gdown 1AF2g2WTtXQwb02S5EAD7aL48c9BkV1ed

### 非台大學生請執行以上程式碼獲取語料 ###

Downloading...
From: https://drive.google.com/uc?id=1AF2g2WTtXQwb02S5EAD7aL48c9BkV1ed
To: /content/dcard-top100.csv
100% 186k/186k [00:00<00:00, 81.8MB/s]


In [ ]:
import os
os.listdir()


['.config', 'dcard-top100.csv', 'sample_data']

In [ ]:
# ✅ STEP 3: 準備你的語料
import pandas as pd

# 直接給出完整路徑
docs_df = pd.read_csv("/content/dcard-top100.csv")

# 把 content 欄位取出變成 list
documents = docs_df["content"].tolist()

# 顯示資料表前幾列確認成功
docs_df.head()


,ID,title,content
0,235443715,三更 21歲這年我差點被活活燒死,部分回應在B117 \r\n謝謝各位的留言，我都有看完\r\n好的不好的，我都接受謝謝大家🙇...
1,235442609,超狂學經歷！195公分帥家教徵學生,https://i.imgur.com/REIEzSd.jpg\r\n\r\n身高195公分...
2,235441998,我的模特界時間管理大師前男友,看過這麼多在Dcard、PTT上的感情渣事和創作文\r\n從沒想過如此荒謬像八點檔的事情居然...
3,235441259,豆皮加爆,剛剛吃小火鍋，跟店員說不要金針菇（怕卡牙縫），於是店員幫我換其他配料..…\r\n\r\n沒...
4,235442693,這樣女生該追嗎,已經約好見面，到了當天晚上七點半才回，我是被耍了嗎 \r\n如下圖\r\n\r\n\r\nh...


In [ ]:
# ✅ STEP 4: 建立 embedding 模型與 FAISS 索引

embedding_model = SentenceTransformer("shibing624/text2vec-base-multilingual")

# 轉換為向量並正規化（便於 cosine 相似度）
doc_embeddings = embedding_model.encode(documents, normalize_embeddings=True)

# 建立向量索引
index = faiss.IndexFlatIP(doc_embeddings.shape[1])
index.add(doc_embeddings)

In [19]:
# 🤖 STEP 5: 呼叫 HuggingFace 模型 API

import requests

API_URL = "https://api-inference.huggingface.co/models/ClueAI/ChatYuan-large-v2"

headers = {"Authorization": f"Bearer {HUGGINGFACE_TOKEN}"}

# 請完成 prompt
def generate_answer(query, context_docs):
    context = "\n".join(f"- {doc[:300]}" for doc in context_docs)  # 避免太長
    prompt = f"""你是一個，請根據以下資料回答問題，若沒有相關資訊請回答「找不到答案」。

資料: {context_docs}

問題: {query}

# 你要問的問題
query = "請問有人在討論感情的困擾嗎？"
query = "大家怎麼看遠距離戀愛？"
query = "中山站有哪些美食？"


# 用前 5 筆資料作為 context
generate_answer(query, context_docs[:5])
generate_answer("有沒有人在講課業壓力？", context_docs[:10])
generate_answer("大家怎麼看遠距離戀愛？", context_docs[:15])
generate_amswer("中山站有哪些美食？", context_docs[:20])





請用一到兩句話回答。
"""

    payload = {"inputs": prompt}
    response = requests.post(API_URL, headers=headers, json=payload, timeout=60)

    try:
        data = response.json()
        if isinstance(data, list) and "generated_text" in data[0]:
            return data[0]["generated_text"]
        elif isinstance(data, dict) and "generated_text" in data:
            return data["generated_text"]
        else:
            return f"⚠️ 無法解析模型回應：{data}"
    except Exception as e:
        return f"❌ 發生錯誤：{e}"



In [20]:
def search(query, top_k=3):

  query_vec = embedding_model.encode([query], normalize_embeddings=True)
  scores, indices = index.search(query_vec, top_k)
  matched_docs = [documents[i] for i in indices[0]]

  return matched_docs


In [23]:
# ✅ STEP 6: 使用者輸入問題，找出最相關的內容

query = "在哪裡可以看到很多草莓？"
query = "有沒有人在討論課業壓力？"
query = "大家怎麼看遠距離戀愛？"
query = "中山站有哪些美食？"

relevant_docs = search(query, top_k=3)
answer = generate_answer(query, relevant_docs)

print("🔎 相關段落：\n", "\n---\n".join(relevant_docs)[:100])
print("\n💬 AI 回答：\n", answer)

NameError: name 'embedding_model' is not defined

## 2. 請回答以下問題 (50%)

請嘗試輸入至少 3 個問題給你的問答系統，並觀察模型回答的準確性與表現：

- 你問了哪些問題？
1. 請問有人在討論感情的困擾嗎？
2. 有沒有人在討論課業壓力？
3. 中山站有哪些美食？

- 模型是否成功從語料中找到與問題相關的內容？
4. 沒有，可能是我的城市有錯誤的地方

- 回答是否合理？是否有亂編的情況？
5. 因為沒有順利找到所以不知道是否有論編

- 若問題無法回答，模型是否能正確回應「找不到答案」？
6. 無法，會跳出錯誤
請根據你的觀察，簡述模型整體表現的優缺點。
7. 模型的優點是可以快速跑過大型語料，讓我們可以省很多力氣。但我要先搞程式的操作，才會更順利。